In [103]:
import os
import json
import copy
import warnings

import sys
sys.path.append("../mpi_scripts/voigt")
from io_utils import strip_comments

In [104]:
def absjoin(*args):
    return os.path.abspath(os.path.join(*args))

In [105]:
def check_config_paths(config):
    for key, value in config.items():
        if isinstance(key, str) and ('filename' in key) and (not os.path.isfile(value)):
            return False, key
        if isinstance(value, dict):
            FLAG, what = check_config_paths(value)
            if not FLAG:
                return FLAG, what
    return True, None

# Synthetic

## top level

In [4]:
dirname_results = "../results/test/"

os.path.abspath(dirname_results)

'/data90t/biomath/pikunov/WORK/ga/mpi_scripts/results/test'

In [5]:
# NEW ga, gamma things

ga_dict = {"seed": 42,
           "n_organisms": int(2**12),
           "n_elites": 32,
           "n_generations": 500,
           "gamma": 0.015,
           "loss": "V_CaT_shared",
           "columns_control": ["V", "fluo"],
           "columns_model": ["V", "fluo"],}

In [6]:
# # test

# ga_dict['n_organisms'] = 32
# ga_dict['n_elites'] = 2
# ga_dict['n_generations'] = 3

In [7]:
dirname_model = "../../models_ctypes/src/model_ctypes/_koivumaki/"

dirname_inputs = "../../models_ctypes/data/koivumaki/fluo_10uM_ghk/"

dirname_states = os.path.join(dirname_inputs, "states/")
dirname_phenotypes = os.path.join(dirname_inputs, "phenotypes/")

filename_so = "koivumaki.so"

model_dict = {"filename_so": absjoin(dirname_model, filename_so),
              "filename_legend_states": absjoin(dirname_model, "legend_states.csv"),
              "filename_legend_constants": absjoin(dirname_model, "legend_constants.csv"),
              
              "t_run": 9,
              "t_sampling": 0.001,
              "tol": 1e-6,
              "stim_period_legend_name": "STIM_PERIOD",}

In [8]:
top_level = dict()
top_level.update(ga_dict)
top_level.update(model_dict)
top_level["output_folder_name"] = absjoin(dirname_results)

In [9]:
top_level

{'seed': 42,
 'n_organisms': 4096,
 'n_elites': 32,
 'n_generations': 500,
 'gamma': 0.015,
 'loss': 'V_CaT_shared',
 'columns_control': ['V', 'fluo'],
 'columns_model': ['V', 'fluo'],
 'filename_so': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/koivumaki.so',
 'filename_legend_states': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_states.csv',
 'filename_legend_constants': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_constants.csv',
 't_run': 9,
 't_sampling': 0.001,
 'tol': 1e-06,
 'stim_period_legend_name': 'STIM_PERIOD',
 'output_folder_name': '/data90t/biomath/pikunov/WORK/ga/mpi_scripts/results/test'}

## experimental conditions

### common

In [102]:
common = {'params': {}}

common_constants_fixed = {# 'Mgi': 1.0,
                          # 'Cao': 2.0,
                          # 'Ko': 4,
                          # 'G_seal': 0,
                          # 'Na_o': 140,
                          }

common_constants_genes_keys = ['STIM_LEVEL', 'PNa', 'gCaL', 'gt', 'gKur',
                               'gK1', 'gKr', 'gKs', 'INaKmax', 'ICaPmax',
                               'gIf', 'kNaCa', 'Jrel_multiplier', 'cpumps',
                               'kSRleak', 'fluo_tot', 'gNab', 'gCab', 'gKb', 'G_seal']

# default_gene_dict = {'bounds': [0.1, 10], 'gamma': 0.015, 'is_multiplier': True}
default_gene_dict = {'bounds': [0.1, 10], 'is_multiplier': True}

common_constants_genes = {x: dict(default_gene_dict) for x in common_constants_genes_keys}

In [11]:
# Custom

common_constants_genes['STIM_LEVEL']['bounds'] = [0.33, 1.33]

common_constants_genes['G_seal']['bounds'] = [0.01, 3]

common_constants_genes['gNab']['bounds'] = [0.01, 3]
common_constants_genes['gCab']['bounds'] = [0.01, 3]
common_constants_genes['gKb']['bounds']  = [0.01, 3]

common['params'].update(common_constants_fixed)
common['params'].update(common_constants_genes)

### CLs

In [12]:
CL_values = [2000, 1000, 500, 333, 250]
# CL_values = [2000, 500, 333, 250]

CL_dict = {}

for CL in CL_values:
    CL_dict[CL] = {}
    
    params_fixed = {'STIM_PERIOD': CL / 1000}
    
    params_genes = {'Nai':  {"bounds": [5, 20]   },
                    'Ki':   {"bounds": [100, 200]},
                    'CaSR': {"bounds": [0.08, 8]}
                   }
    
    CL_dict[CL]['params'] = {**params_fixed, **params_genes}
    
    CL_dict[CL]['filename_phenotype'] = absjoin(dirname_phenotypes, f'phenotype_{CL}.csv')
    CL_state = 1000
    CL_dict[CL]['filename_state'] = absjoin(dirname_states, f'state_{CL_state}.csv')
    
#     import pandas as pd
#     import numpy as np
#     f_csv = CL_dict[CL]['filename_state']
#     f_txt = f_csv.replace(".csv", ".txt")
#     np.savetxt(f_txt, pd.read_csv(f_csv).iloc[:, -1].values)
#     CL_dict[CL]['filename_state'] = f_txt

# f_txt

In [13]:
config = top_level
config['experimental_conditions'] = {'common': common}
config['experimental_conditions'].update(CL_dict)

In [15]:
if not (pair := check_config_paths(config))[0]:
    msg = "Check this: {}".format(pair[1])
    warnings.warn(msg)
else:
    with open("../mpi_scripts/voigt/configs/config.json", 'w') as f:
        f.write(json.dumps(config, indent=4))
    print("Config is done!")

Config is done!


In [15]:
# with open("../../chimera/ghk_gav1/configs/config.json", 'w') as f:
#     f.write(json.dumps(config, indent=4))

# Voigt

## top level

In [129]:
dirname_configs = "../../mpi_scripts/mpi_scripts/voigt/configs/configs_voigt/"
os.makedirs(dirname_configs, exist_ok=True)

In [130]:
dirname_results = "../results/voigt/"

os.path.abspath(dirname_results)

'/data90t/biomath/pikunov/WORK/ga/mpi_scripts/results/voigt'

In [131]:
# NEW ga, gamma things

ga_dict = {"seed": 42,
           "n_organisms": int(2**12),
           "n_elites": 32,
           "n_generations": 500,
           "gamma": 0.015,
           "loss": "V_CaT_shared",
           "columns_control": ["V", "CaT"],
           "columns_model": ["V", "fluo"],
           "column_stim_protocol": "Ist",}

In [132]:
# # test

# ga_dict['n_organisms'] = 32
# ga_dict['n_elites'] = 2
# ga_dict['n_generations'] = 3

In [133]:
dirname_model = "../../models_ctypes/src/model_ctypes/_koivumaki/"

dirname_states = "../../models_ctypes/data/koivumaki/fluo_10uM_ghk/states/"

filename_so = "koivumaki.so"

model_dict = {"filename_so": absjoin(dirname_model, filename_so),
              "filename_legend_states": absjoin(dirname_model, "legend_states.csv"),
              "filename_legend_constants": absjoin(dirname_model, "legend_constants.csv"),
              
              "t_run": 9,
              "t_sampling": 0.0001,
              "tol": 1e-6,
              "stim_period_legend_name": "STIM_PERIOD",}

In [134]:
top_level = dict()
top_level.update(ga_dict)
top_level.update(model_dict)
top_level["output_folder_name"] = absjoin(dirname_results)

In [135]:
top_level

{'seed': 42,
 'n_organisms': 4096,
 'n_elites': 32,
 'n_generations': 500,
 'gamma': 0.015,
 'loss': 'V_CaT_shared',
 'columns_control': ['V', 'CaT'],
 'columns_model': ['V', 'fluo'],
 'column_stim_protocol': 'Ist',
 'filename_so': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/koivumaki.so',
 'filename_legend_states': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_states.csv',
 'filename_legend_constants': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_constants.csv',
 't_run': 9,
 't_sampling': 0.0001,
 'tol': 1e-06,
 'stim_period_legend_name': 'STIM_PERIOD',
 'output_folder_name': '/data90t/biomath/pikunov/WORK/ga/mpi_scripts/results/voigt'}

## experimental conditions

### common

In [136]:
common = {'params': {}}

common_constants_fixed = {#'Mgi': 1.0,
                          'Cao': 2.0,
                          'Ko': 4,
                          # 'G_seal': 0,
                          'Nao': 140,
                          }

common_constants_genes_keys = ['STIM_LEVEL', 'PNa', 'gCaL', 'gt', 'gKur',
                               'gK1', 'gKr', 'gKs', 'INaKmax', 'ICaPmax',
                               'gIf', 'kNaCa', 'Jrel_multiplier', 'cpumps',
                               'kSRleak', 'fluo_tot', 'gNab', 'gCab', 'gKb', 'G_seal']

# default_gene_dict = {'bounds': [0.1, 10], 'gamma': 0.015, 'is_multiplier': True}
default_gene_dict = {'bounds': [0.1, 10], 'is_multiplier': True}

common_constants_genes = {x: dict(default_gene_dict) for x in common_constants_genes_keys}

In [137]:
# Custom

common_constants_genes['STIM_LEVEL']['bounds'] = [0.33, 1.33]

common_constants_genes['G_seal']['bounds'] = [0.01, 3]

common_constants_genes['gNab']['bounds'] = [0.01, 3]
common_constants_genes['gCab']['bounds'] = [0.01, 3]
common_constants_genes['gKb']['bounds'] = [0.01, 3]

common['params'].update(common_constants_fixed)
common['params'].update(common_constants_genes)

### Group / Cell / CL

In [138]:
filename_phenotype_template = "../../data/voigt/10kHz/G{ig}/C{ic}/phenotype_{CL}.csv"
filename_protocol_template = "../../data/voigt/10kHz/G{ig}/C{ic}/protocol_{CL}.csv"

In [139]:
def lr(i):
    return list(range(1, i + 1))

cells = [lr(3), lr(4), lr(5), lr(8)]

In [140]:
CL_values_states = [2000, 1000, 500, 333, 250]
CL_values_phenotypes = [2064, 1032, 516, 344, 258]

In [141]:
for ig in range(1, 4 + 1):
    for ic in cells[ig - 1]:

        CL_dict = {}

        for CL_state, CL_phen in zip(CL_values_states, CL_values_phenotypes):

            CL_dict[CL_phen] = {}

            params_fixed = {'STIM_PERIOD': CL_phen / 1000}

            params_genes = {'Nai':  {"bounds": [5, 20]   },
                            'Ki':   {"bounds": [100, 200]},
                            'CaSR': {"bounds": [0.08, 8]}
                           }

            CL_dict[CL_phen]['params'] = {**params_fixed, **params_genes}

            CL_dict[CL_phen]['filename_phenotype'] = absjoin(filename_phenotype_template.format(ig=ig, ic=ic, CL=CL_phen))
            CL_dict[CL_phen]['filename_state'] = absjoin(dirname_states, f'state_{CL_state}.csv')
            CL_dict[CL_phen]['filename_stim_protocol'] = absjoin(filename_protocol_template.format(ig=ig, ic=ic, CL=CL_phen))

            
            config = top_level
            config['experimental_conditions'] = {'common': common}
            config['experimental_conditions'].update(CL_dict)
            
            if not (pair := check_config_paths(config))[0]:
                msg = "Check this: {}".format(pair[1])
                warnings.warn(msg)
                break
            else:
                filename_config = os.path.join(dirname_configs, "config_G{ig}C{ic}.json".format(ig=ig, ic=ic))
                with open(filename_config, 'w') as f:
                    f.write(json.dumps(config, indent=4))
                # print("Config is done!")

In [142]:
# with open("../../chimera/ghk_gav1/configs/config.json", 'w') as f:
#     f.write(json.dumps(config, indent=4))

In [143]:
config

{'seed': 42,
 'n_organisms': 4096,
 'n_elites': 32,
 'n_generations': 500,
 'gamma': 0.015,
 'loss': 'V_CaT_shared',
 'columns_control': ['V', 'CaT'],
 'columns_model': ['V', 'fluo'],
 'column_stim_protocol': 'Ist',
 'filename_so': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/koivumaki.so',
 'filename_legend_states': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_states.csv',
 'filename_legend_constants': '/data90t/biomath/pikunov/WORK/ga/models_ctypes/src/model_ctypes/_koivumaki/legend_constants.csv',
 't_run': 9,
 't_sampling': 0.0001,
 'tol': 1e-06,
 'stim_period_legend_name': 'STIM_PERIOD',
 'output_folder_name': '/data90t/biomath/pikunov/WORK/ga/mpi_scripts/results/voigt',
 'experimental_conditions': {'common': {'params': {'Cao': 2.0,
    'Ko': 4,
    'Nao': 140,
    'STIM_LEVEL': {'bounds': [0.33, 1.33], 'is_multiplier': True},
    'PNa': {'bounds': [0.1, 10], 'is_multiplier': True},
    'gCaL': {'bounds': [0.1, 1